# Email Topic Classification

Train various classifiers to distinguish between topics based on their text. Each document will be represented with by a "bag-of-words" model resulting in sparse feature vectors. 

In [2]:
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import sklearn

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

pd.options.display.max_rows = 100
print sklearn.__version__
print np.__version__

0.19.2
1.15.3


Load the data and strip out the metadata so that the calssifiers only use textual features. By default, newsgroups data is split into train and test sets. I will further split the test so I have a dev set. For simplicity, only using four categories from the set for this project

In [ ]:
labels = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=labels)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=labels)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

## Print out the message and label for each of the first 5 documents

In [ ]:
num_examples=5
names = newsgroups_train['target_names']
for n in range(num_examples):
    print '\nSample number:', n
    print 'Classification Label:', names[train_labels[n]]
    print '\nText:\n', train_data[n]

## Extracting the text features using CountVectorizer()
Getting to know the CountVectorizor() function through a number of steps. Fit and transform the data, then look at:  
> (a) The size of the vocabulary, the average number of non-zero features per example, the fraction of the entries in the matrix that are non-zero  
(b) The first and last feature strings of the vocabulary in alphabetical order  
(c) Find the average number of non-zero features per example using a vocabulary of just the main words in each label ["atheism", "graphics", "space", "religion"]  
(d) Look at the size of the vocabulary using bigram and trigam cahracter features instead of unigram word features  
(e) Look at the size of the vocabulary after pruning words that appear in less than 10 documents  
(f) with default settings to CountVectorizer, what fraction of words in the dev data are missing from the training data vocabulary?

In [ ]:
## PART A
print 'PART (A):'

# vectorize the features, fit_transform them into a sparse matrix
v = CountVectorizer()
features = v.fit_transform(train_data)
print '\nSize of vocabulary:', len(v.vocabulary_)

# find the mean number of nonzero features per example
avg_nz = 1.0 * np.sum([f.nnz for f in features]) / features.shape[0]
print 'Average number of non-zero features per example:', avg_nz

# find the fraction of nonzero entries in the features matrix
frac_nz = 1.0 * features.nnz / features.toarray().size
print 'Fraction of entries that are non-zero:', frac_nz

## PART B
print '\nPART (B):'

# get the first and last feature strings
names = v.get_feature_names()
print '\n0th feature: {}, last feature: {}'.format(names[0], names[-1])

## PART C
print '\nPART (C):'
# specify the vocab and fit_transform the training data
v2 = CountVectorizer(vocabulary=['atheism', 'graphics', 'space', 'religion'])
features2 = v2.fit_transform(train_data)

# find the mean number of nonzero features per document
avg_nz = 1.0 * np.sum([f.nnz for f in features2]) / features2.shape[0]
print '\nVector Shape:', features2.shape
print 'Average number of non-zero features per example:', avg_nz

## PART D
print '\nPART (D):'

# extract bigram and trigram character features
v3 = CountVectorizer(analyzer='char', ngram_range=(2, 3))
v3.fit_transform(train_data)

# get the size of the vocabulary
n = len(v3.vocabulary_)
print '\nSize of vocabulary using bigrams and trigrams:', n

## PART E
print '\nPART (E):'

# change min_df so doc frequency is >= 10 total documents
v4 = CountVectorizer(min_df=1.0 * 10/len(train_data))
v4.fit_transform(train_data)
print '\nSize of vocabulary appearing in >= 10 docs:', len(v4.vocabulary_)

## PART F
print '\nPART (F):'

# get the vocab of the training data and the dev data
v = CountVectorizer()
v.fit_transform(train_data)
train_vocab = v.get_feature_names()
v.fit_transform(dev_data)
dev_vocab = v.get_feature_names()

# find the fraction of words in dev data vocab that are not in the 
# training data vocab
diff = 1.0 * len(set(dev_vocab).difference(set(train_vocab))) / len(train_vocab)
print '\nFraction of words in dev_data and not in train_data vocab:', diff

## Compare simplistic versions of three classification models
Find the optimal value for `k` in using a kNN model, then find the optimal value for `alpha` using a Multinomial Naive Bayes model, then find the optimal value for the regularization stregnth `C` in a Logistic Regression model. Look at the relationship between the weight vector for each class vs. the sum squared weight values for each setting of the regularization strength `C`. 

In [ ]:
def c_plotter(start, f, train_labels, labels, c_vals):
    ssq_total = []
#     for c in np.arange(1,100,2):
    for c in c_vals:

        # fit a logistic regression model with regularization strength c
        lg = LogisticRegression(C=c)
        lg.fit(f, train_labels)

        # Take the sum of the squared weights for each label for each 
        # value of c and append it to a list
        weights = lg.coef_
        ssq = np.sum(weights**2, axis=1)
        ssq_total.append(ssq)
    # convert to a numpy array
    ssq_total = np.array(ssq_total)
    print 'runtime:', time.time() - start

    # plot the regularization strength vs. the sum of the squared
    # weights for each label
    plt.figure(figsize=(12,8))
    for i in range(4):
        plt.plot(c_vals, ssq_total[:,i])
    t = (
        'Regularization Strength vs. Sum Squared Feature Weights\n' + 
        'C value range: min = {},'.format(c_vals[0]) + 
        ' max = {},'.format(c_vals[-1]) + 
        ' step size = {}'.format(c_vals[1] - c_vals[0])
    )
    plt.title(t)
    plt.xlabel('Regularization Strength (C)')
    plt.ylabel('Sum Squared Feature Weights')
    plt.legend([labels[0], labels[1], labels[2], labels[3]])

    
v = CountVectorizer()
f = v.fit_transform(train_data)
scorer = metrics.make_scorer(metrics.f1_score, average='weighted')

## k-NN classifier
# Use kNN classifier and optimize the number of neighbors using grid 
# search with the f1-score as the scoring metric
print '\nK-NEAREST NEIGHBORS MODEL:\n'
start = time.time()

# n_neighbors was tested over a much larger range, but the optimal 
# value was found in the range below, so I shortened the search 
# range to improve runtime
params = {'n_neighbors': np.arange(90, 100)}
gs = GridSearchCV(KNeighborsClassifier(), params, scoring=scorer)
gs.fit(f, train_labels)
end = time.time()
print 'Best k:', gs.best_params_.values()[0]
print 'Best F1-score:', gs.best_score_ 
print 'runtime:', round(end-start, 3)

## Multinomial NB
# Use the multinomial Naive Bayes classifier and optimize the alpha 
# value using grid search with the f1-score as the scoring metric
print '\nMULTINOMIAL NAIVE BAYES MODEL:\n'
start = time.time()

# alpha was tested over a much larger range, but the optimal 
# value was found in the range below, so I shortened the search 
# range to improve runtime
params = {'alpha': np.arange(0.0, 0.01, 0.0001)}
gs = GridSearchCV(MultinomialNB(), params, scoring=scorer)
gs.fit(f, train_labels)
end = time.time()
print 'Best alpha:', gs.best_params_.values()
print 'Best F1-score:', gs.best_score_ 
print 'runtime:', round(end-start, 3)

## Logistic Regression
# Use logistic regression and optimize the regularization strength
# using grid search with the f1-score as the scoring metric
print '\nLOGISTIC REGRESSION MODEL:\n'
start = time.time()

# regularization strength was tested over a much larger range, but 
# the optimal value was found in the range below, so I shortened the
# search range to improve runtime
params = {'C': np.arange(0.1, .2, 0.01)}
gs = GridSearchCV(LogisticRegression(), params, scoring=scorer)
gs.fit(f, train_labels)
end = time.time()
print 'Best regularization parameter:', gs.best_params_.values()
print 'Best F1-score:', gs.best_score_ 
print 'runtime:', round(end-start, 3)

## Part C: 
# describe the relationship between accuracy and the sum of the 
# squared weights
labels = newsgroups_train['target_names']
start = time.time()
c_plotter(start, f, train_labels, labels, c_vals=np.arange(0.001,1,0.01))
start = time.time()
c_plotter(start, f, train_labels, labels, c_vals=np.arange(1, 100, 2))

The nearest neighbors model doesn't work very well in this scenario because there aren't any neighbors that are very near. By this I mean that the vectorized features are a sparse matrix so a large majority of the data that is used as nearest neighbors will be empty or at least not contain meaningful information. This is why the optimal number of neighbors was so high at 96, because we need to include that many points in the sparse matrix to have a few points of meaningful information
    
As discussed here https://medium.com/@sangha_deb/naive-bayes-vs-logistic-regression-a319b07a5d4c, The reason logistic regression does not work as well as Naive Bayes for this data set is likely due to a relatively small sample size. Both Naive Bayes and Logistic Regression will reach an asymptotic solution of predictive accuracy as sample size increases, but Naive Bayes will reach that asymptotic solution faster due to operating at O(log(n)) as compared to Logistic Regression with O(n). So the training set is 2034 samples and this is apparently not enough for Logistic Regression to win out. If we had 10,000 or 100,000 samples, Logistic Regression would likely perform better.
    
The relationship between the regularization strength, C, and the sum of squared weights is somewhat logarithmic. This makes sense because the regularization strength is a penalty to the total size of the weight to ensure that the sum squared weight does not become too large and thus overfit the data. So if the regularization is growing linearly, then the sum squared weights must grow less than linearly, but will still be growing, and this shows in the curve.

## Comparing weights of top 5 words for each label
Determine the five words with the largest weights for each of the four labels for 20 words in total. Create a table showing the weight of each of these freatures for each of the four labels. Create the same table using bigram features only, then repeat using unigram and bigram features

In [ ]:
'''
The below function generates a word map which maps the top 5 words to 
their index in the logistic regression coefficient matrix. Then using 
those indices, find the coefficient/weight of each word for each 
label. The function returns a pandas DataFrame with the rownames as 
the array of top 5 words for each label, the columns names as each 
label, and the entries as the weight of that word for each label
'''
def top5_table(train_data, train_labels, grams=(1,1), num_words=5):
    
    # get the target labels
    labels = newsgroups_train['target_names']
    
    # vectorize the data and extract the vocabulary
    v = CountVectorizer(ngram_range=grams)
    f = v.fit_transform(train_data)
    vocab = v.vocabulary_
    
    # fit the logisitic regression to extract the coefficients (weights) 
    # of the feature vectors
    lg = LogisticRegression(C=0.18)
    lg.fit(f, train_labels)
    
    # generate the weight map
    word_map = {}
    words = []
    for ind,c in enumerate(lg.coef_):
        
        # find the top 5 words by weight for each label.
        top5 = np.argsort(c)[::-1][:num_words]
        words = [(k, v) for k,v in vocab.items() if v in top5]
        
        # store top 5 for each label in single word map
        for w in words:
            word_map[w[0]] = w[1]
    
    # grab the weights at the indices in the word map
    weight_arr = np.array([
        [lg.coef_[i][v] for v in word_map.values()] 
        for i in range(len(labels))
    ])
    
    # generate the DataFrame
    df = pd.DataFrame(weight_arr.T, columns=labels, index=word_map.keys())
    
    return df

### STUDENT END ###
df1 = top5_table(train_data, train_labels)
df2 = top5_table(train_data, train_labels, grams=(2,2))
df3 = top5_table(train_data, train_labels, grams=(1,2))
print '\nUnigram features only:\n'
print df1
print '\nBigram features only:\n'
print df2
print '\nUnigram and Bigram features:\n'
print df3

For each label, the 5 words that have the strongest weight for that label carry very strong positive coefficients which is to be expected. What is interesting is that the other 15 words that best represent the other 3 labels carry strong negative coefficients for that label, which indicates that the regression has identified the strong indicators for each label and weighted them negatively for every other label which helps to reinforce the classification. Additionally, the use of bigrams only chooses some interesting selections as the most important words for each label. With unigrams only, the words that were selected made good logical sense that they would carry strong weight for that label (e.g. 'nasa' for sci.space, 'christian' for talk.religion.misc, and 'atheism' for alt.atheism). The bigram features are harder to logically justify. Why is 'cheers kent' the strongest indicator of the label alt.atheism? How is 'and such' a strong indicator of sci.space? In running the regression with both unigrams and bigrams, it becomes apparent that these aren't actually that strong of indicators as the 20 word list in this regression contains nearly the same words as it does when using just unigrams, indicating that the bigram features do not influence the classification as strongly as the unigram features

## Improving the regression classifier
pass a custom preprocessor to CountVectorizer

In [ ]:
def empty_preprocessor(s):
    return s

def better_preprocessor(s):
    # make everything lowercase
    s = s.lower()
    
    # replace standalone numeric strings with the token "numeric_string"
    s = re.sub(r"\b\d+\b", "numeric_string", s)
    
    # remove all apostrophes
    s = re.sub(r"\'", "", s)
    
    # replace all email addresses with token "email_string"
    s = re.sub(r"\b\w+@\w+[\.\w+]+\b", "email_string", s)
    
    # replace words less than 5 letters with token "lt_five"
    s = re.sub(r"\b\w{1,4}\b", 'lt_five', s)
    
    return s

# using the empty_preprocessor
v = CountVectorizer(preprocessor=empty_preprocessor)
#     v = CountVectorizer()
f = v.fit_transform(train_data)
vocab1 = v.vocabulary_
d = v.transform(dev_data)
lg = LogisticRegression(C=0.18)
lg.fit(f, train_labels)
pred = lg.predict(d)
print '\nF1-score with empty preprocessor:', 
print metrics.f1_score(dev_labels, pred, average='weighted')
print 'Size of vocab with empty preprocessor:', len(vocab1.keys())

# using the better_preprocessor
v = CountVectorizer(preprocessor=better_preprocessor)
f = v.fit_transform(train_data)
vocab2 = v.vocabulary_
d = v.transform(dev_data)
lg = LogisticRegression(C=0.18)
lg.fit(f, train_labels)
pred = lg.predict(d)
print '\nF1-score with better preprocessor:', 
print metrics.f1_score(dev_labels, pred, average='weighted')
print 'Size of vocab with better preprocessor:', len(vocab2.keys())

### STUDENT END ###

## Pruning the vocabulary using "l1" penalty
Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. Prune the vocabulary to only those words with at least one non-zero weight when using "l1" penalty, and then retrain the model using "l2" penalty. Then plot the accuracy of the re-trained model vs. the vocabulary size after pruning unused features while adjusting the regularization parameter `C`

In [ ]:
# Keep this random seed here to make comparison easier.
np.random.seed(0)

c_vals = np.arange(0.001,1,0.01)
acc = []
vocab_size = []
for c in c_vals:
    # step 1: build the L1 model
    v = CountVectorizer()
    f = v.fit_transform(train_data)
    vocab = v.vocabulary_
    lg = LogisticRegression(penalty='l1', tol=0.01, C=c)
    lg.fit(f, train_labels)

    # step 2: prune the vocab
    i,j = np.nonzero(lg.coef_)
    vocab_pruned = {k:v for k,v in vocab.items() if v in j}
    vocab_size.append(len(vocab_pruned.keys()))

    # step 3: train an L2 model with pruned vocab
    v = CountVectorizer(vocabulary=vocab_pruned.keys())
    f = v.fit_transform(train_data)
    p = v.transform(dev_data)
    lg = LogisticRegression(penalty='l2', tol=0.01, C=c)
    lg.fit(f, train_labels)
    pred = lg.predict(p)
    acc.append(metrics.accuracy_score(dev_labels, pred))

plt.figure(figsize=(12,8))
plt.plot(vocab_size, acc, 'o-')
plt.xlabel('Size of Pruned Vocabulary')
plt.ylabel('Logistic Regression Accuracy using L2 Penalty')

start = time.time()
print 'runtime:', time.time() - start
### STUDENT END ###

## Compare the TfidfVectorizer to the CountVectorizer
Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:
> maximum predicted probability / predicted probability of the correct label

Determine what mistakes the classifier is making by looking at the top 20 words for each label

In [ ]:
num_examples = 3
    
# build the tfidf transform and fit the training data and the dev
# dev data
t = TfidfVectorizer()
f = t.fit_transform(train_data)
d = t.transform(dev_data)

# Fit a logistic regression model to the vectorized training data 
# and predict the probabilities
lg = LogisticRegression(C=100)
lg.fit(f, train_labels)
pred = lg.predict(d)
probs = lg.predict_proba(d)

# find the maximum probabilities
max_probs = np.max(probs, axis=1)

# find the proabilities of the correct label
correct_probs = np.array([probs[i,dev_labels[i]] for i in range(len(dev_labels))])
R = max_probs / correct_probs

# display the highest R ratio examples
labels = newsgroups_train['target_names']
top = np.argsort(R)[::-1][:num_examples]
for i in range(num_examples):
    print "\nSample:", i
    print 'True Label:', labels[dev_labels[top[i]]]
    print 'Predicted Label:', labels[pred[top[i]]]
    print 'R ratio:', R[top[i]]
    print '\n', dev_data[top[i]]

In [ ]:
def top_table(train_data, train_labels, grams=(1,1), num_words=5):
    
    # get the target labels
    labels = newsgroups_train['target_names']
    
    # vectorize the data and extract the vocabulary
    # v = CountVectorizer(ngram_range=grams)
    v = TfidfVectorizer(ngram_range=grams)
    f = v.fit_transform(train_data)
    vocab = v.vocabulary_
    
    # fit the logisitic regression to extract the coefficients (weights) 
    # of the feature vectors
    lg = LogisticRegression(C=0.18)
    lg.fit(f, train_labels)
    
    # generate the weight map
    word_map = {}
    words = []
    for ind,c in enumerate(lg.coef_):
        
        # find the top 5 words by weight for each label.
        top5 = np.argsort(c)[::-1][:num_words]
        words = [(k, v) for k,v in vocab.items() if v in top5]
        
        # store top 5 for each label in single word map
        for w in words:
            word_map[w[0]] = w[1]
    
    # grab the weights at the indices in the word map
    weight_arr = np.array([
        [lg.coef_[i][v] for v in word_map.values()] 
        for i in range(len(labels))
    ])
    
    # generate the DataFrame
    df = pd.DataFrame(weight_arr.T, columns=labels, index=word_map.keys())
    
    return df

df1 = top_table(train_data, train_labels, num_words=20)
print '\nUnigram features only:\n'
print df1

To understand what mistakes are being made here, I modified the function in porblem 4 to use the TfidfVectorizer instead of the CountVectorizer and I looked at the 20 highest frequency words for each label. In the first example, there are many words present that are only associated with the comp.graphics category, such as "Apple", "Microsoft", "ASCII", "LaTeX", etc. Additionally, the word "file" appears many times thorughout the document and is strongly associated with the comp.graphics label and very negatively associated with the talk.religion.misc label. So the vectorizer takes these words out of context and the regression just looks at the word frequency and there is not enough intelligence to know that these words that are strongly associated with comp.graphics are used here to promote a religious text.  
  
The second example does the same thing. The word "anyone" is one of the top 20 words in the comp.graphics category and words like "ftp" and "internet" are only positively associated with the comp.graphics label and since there are very little words in this document, these few words outweigh "Mormon".  
  
The third example has very few words to analyze, but interestingly words such as "of", "the" and "children" were strongly associated with talk.religion.misc and so since there isn't anything stronger to go off of here, those words are probably the key predictors in classifying this document.  

Since the TfidfVectorizer just counts the frequency of each word, common short words such as "is", "of", "at", etc. are being given very strong weights in the classification even though they don't actually contain a lot of contextual meaning. One way to improve this would be to change how we define tokens in the vectorizer. If we only looked at words of length 5 letters or more for example, this will put heavier weight on more important words to the context of the document

## Improve the classifier by tokenizing on 5 or more letter words

In [ ]:
# testing the original vectorizer with default two-letter token
t = TfidfVectorizer()
lg = LogisticRegression()
f = t.fit_transform(train_data)
d = t.transform(dev_data)
lg.fit(f, train_labels)
pred = lg.predict(d)
skore = metrics.f1_score(dev_labels, pred, average='weighted')
print 'F1-Score using two or more letters:', skore

# testing the vecotrizer with five-letter or more tokens
t = TfidfVectorizer(token_pattern=u'(?u)\\b\\w{5,}\\b')
lg = LogisticRegression()
f = t.fit_transform(train_data)
d = t.transform(dev_data)
lg.fit(f, train_labels)
pred = lg.predict(d)
skore = metrics.f1_score(dev_labels, pred, average='weighted')
print 'F1-Score using five or more letters:', skore